# Question 1

In [36]:
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)

In [37]:
df_postcodes = dfs[0]
df_postcodes = df_postcodes[df_postcodes['Borough'] != 'Not assigned']
df_postcodes = df_postcodes.reset_index().drop('index', axis = 1)
print("The amount of neighbourhoods with a 'Not assigned' value is: " + str(len(df_postcodes[df_postcodes['Neighbourhood'] == 'Not assigned'])))

The amount of neighbourhoods with a 'Not assigned' value is: 0


In [34]:
postcode_counts = {}
for i in range(len(df_postcodes['Postal Code'])):
    if df_postcodes.iloc[i]['Postal Code'] in postcode_counts:
        postcode_counts[df_postcodes.iloc[i]['Postal Code']] += 1
    else:
        postcode_counts[df_postcodes.iloc[i]['Postal Code']] = 1
for i in postcode_counts:
    if postcode_counts[i] > 1:
        print('There is one postal code that is duplicate and dataframe needs to be adjusted.')
        break
    elif i == 'M8Z' and postcode_counts[i] == 1:
        print('There are no duplicate postal codes in the dataframe.')

There are no duplicate postal codes in the dataframe.


In [38]:
df_postcodes

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [39]:
df_postcodes.shape

(103, 3)

# Question 2

In [41]:
import geocoder

In [54]:
lat = []
lng = []
for j in range(len(df_postcodes['Postal Code'])):
    coords = None
    while coords is None:
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(df_postcodes.iloc[j]['Postal Code']))
        coords = g.latlng
    lat.append(coords[0])
    lng.append(coords[1])

In [57]:
df_postcodes['Latitude'] = lat

In [58]:
df_postcodes['Longitude'] = lng

In [60]:
df_postcodes

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


In [61]:
df_postcodes.shape

(103, 5)

# Question 3

**I decided to just analyze boroughs with 'Toronto' in it, so the following lines of code are reducing the main dataframe to only those boroughs**

In [63]:
Toronto_postcodes = []
for k in range(len(df_postcodes['Postal Code'])):
    if 'Toronto' in df_postcodes.iloc[k]['Borough']:
        Toronto_postcodes.append(df_postcodes.iloc[k]['Postal Code'])

In [66]:
df_tor = [df_postcodes.iloc[m] for m in range(len(df_postcodes['Postal Code'])) if df_postcodes.iloc[m]['Postal Code'] in Toronto_postcodes]

In [72]:
df_tor = pd.DataFrame(df_tor)
df_tor.reset_index(drop = True, inplace = True)
df_tor.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M4E,East Toronto,The Beaches,43.67709,-79.29547


**Below is code to retrieve all the nearby venues for the boroughs in the df_tor dataframe**

In [77]:
import requests
import json

In [74]:
CLIENT_ID = 'HGE25U5TNWONG0CX5I004L34WU5PU2II01DFTRIKCQDXQALZ'
CLIENT_SECRET = '2S0EZ4ND3BZ4SIR2NKDEPF1ZNFN3RIFVHWRKPIDL0N5IHFXE'
VERSION = '20201117'
LIMIT = 100

In [75]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [78]:
toronto_venues = getNearbyVenues(df_tor['Neighbourhood'], df_tor['Latitude'], df_tor['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [86]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


**Below is code to get the dummies for the various venue types for each neighborhood that is then grouped up and the mean for each neighborhood is found**

In [120]:
tor_vens_dum = pd.get_dummies(toronto_venues[['Venue Category']], prefix_sep = '', prefix = '')

tor_vens_dum['Neighborhood'] = toronto_venues['Neighborhood']
tor_vens_dum.shape
fixed_columns = ['Neighborhood'] + list([x for x in tor_vens_dum.columns if x != 'Neighborhood'])
tor_vens_dum = tor_vens_dum[fixed_columns]
tor_vens_dum.head()

,Neighborhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [143]:
tor_grouped = tor_vens_dum.groupby('Neighborhood').mean().reset_index()

In [144]:
from sklearn.cluster import KMeans
tor_grouped_new = tor_grouped.drop('Neighborhood', axis = 1)

In [145]:
kmeans = KMeans(n_clusters=4, random_state=1).fit(tor_grouped_new)

In [210]:
labels = kmeans.labels_
tor_vens_joined = pd.DataFrame(tor_grouped['Neighborhood'])
tor_vens_joined['Label'] = labels
lats = []
lngs = []
for n in tor_vens_joined['Neighborhood']:
    for o in range(len(toronto_venues['Neighborhood'])):
        if n == toronto_venues.iloc[o]['Neighborhood']:
            lats.append(toronto_venues.iloc[o]['Neighborhood Latitude'])
            lngs.append(toronto_venues.iloc[o]['Neighborhood Longitude'])
            break
tor_vens_joined['Latitude'] = lats
tor_vens_joined['Longitude'] = lngs
tor_vens_joined = tor_vens_joined.join(tor_grouped_new)
tor_vens_joined.head()

,Neighborhood,Label,Latitude,Longitude,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,3,43.64536,-79.37306,0.000000,0.0,0.00,0.016667,0.0,0.016667,...,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.016667
1,"Brockton, Parkdale Village, Exhibition Place",3,43.63941,-79.42676,0.011494,0.0,0.00,0.000000,0.0,0.000000,...,0.000000,0.011494,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.011494
2,"Business reply mail Processing Centre, South C...",3,43.64869,-79.38544,0.000000,0.0,0.02,0.000000,0.0,0.010000,...,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.0,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",3,43.64082,-79.39818,0.000000,0.0,0.00,0.000000,0.0,0.000000,...,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.013158
4,Central Bay Street,3,43.65609,-79.38493,0.000000,0.0,0.00,0.000000,0.0,0.013158,...,0.000000,0.000000,0.000000,0.013158,0.013158,0.013158,0.0,0.0,0.0,0.000000


**Below is the map for the different clusters that were found withe KMeans method of clustering similar areas according to venue type**

In [217]:
import folium


In [268]:
map_clusters = folium.Map(location = [43.6532, -79.3832], zoom_start = 12)
colors = ['#ff5733', '#0a8717', '#d33fee', '#17afe8']

for x in range(len(tor_vens_joined['Label'])):
    lat = tor_vens_joined.iloc[x]['Latitude']
    lng = tor_vens_joined.iloc[x]['Longitude']
    neigh = tor_vens_joined.iloc[x]['Neighborhood']
    cluster = tor_vens_joined.iloc[x]['Label']
    label = folium.Popup(str(neigh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(location=[lat,lng],radius=5,color=colors[cluster],fill=True,fill_color=colors[cluster],popup=label).add_to(map_clusters)
    
map_clusters

map_clusters

**Below is code to analyze similarities between clusters**

In [223]:
tor_vens_sim = tor_vens_joined.drop(['Latitude','Longitude'], axis = 1)
tor_vens_sim = tor_vens_sim.groupby('Label').mean()

In [224]:
tor_vens_sim

,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Label,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000371,0.000408,0.009516,0.00117,0.001112,0.006474,0.000424,0.003218,0.007395,0.004545,...,0.001626,0.008264,0.000424,0.001344,0.004292,0.004323,0.000556,0.000597,0.000323,0.004884


In [235]:
sim_list = []
for i in range(len(tor_vens_sim['Accessories Store'])):
    values = []
    for j in tor_vens_sim.iloc[i]:
        if j != 0:
            values.append(j)
    values.sort(reverse = True)
    if len(values) > 5:
        for k in values[0:5]:
            for l in tor_vens_sim:
                if k == tor_vens_sim[l][i]:
                    sim_list.append(l)
    else:
        for k in values[0:len(values)]:
            for l in tor_vens_sim:
                if k == tor_vens_sim[l][i]:
                    sim_list.append(l)
    sim_list.append('END')
sim_list

['Playground',
 'Park',
 'Trail',
 'Gym',
 'Gym Pool',
 'Gym',
 'Gym Pool',
 'END',
 'Bus Line',
 'Swim School',
 'Bus Line',
 'Swim School',
 'END',
 'French Restaurant',
 'Park',
 'French Restaurant',
 'Park',
 'END',
 'Coffee Shop',
 'Café',
 'Park',
 'Restaurant',
 'Italian Restaurant',
 'END']

In [264]:
df_sim = pd.DataFrame(data = ' ', index = [0,1,2,3], columns = ['Top Venues'])
cluster = 0
for i in sim_list:
    if i != 'END' and df_sim.loc[cluster]['Top Venues'] != ' ' and i not in df_sim.loc[cluster]['Top Venues']:
        df_sim.loc[cluster]['Top Venues'] = df_sim.loc[cluster]['Top Venues'] + ', ' + str(i)
    elif i in df_sim.loc[cluster]['Top Venues']:
        pass
    elif i != 'END':
        df_sim.loc[cluster]['Top Venues'] = str(i)
    else:
        cluster += 1
df_sim = df_sim.reset_index()
df_sim.rename(columns={'index':'Cluster'})

,Cluster,Top Venues
0,0,"Playground, Park, Trail, Gym, Gym Pool"
1,1,"Bus Line, Swim School"
2,2,"French Restaurant, Park"
3,3,"Coffee Shop, Café, Park, Restaurant, Italian R..."


In [263]:
tor_vens_joined.groupby('Label').count()

,Neighborhood,Latitude,Longitude,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
Label,,,,,,,,,,,,,,,,,,,,,
0,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,31,31,31,31,31,31,31,31,31,31,...,31,31,31,31,31,31,31,31,31,31


**Observations**
<ol>
    <li>Cluster 0 has five neighborhoods and is comprised of active venues such as playgrounds and parks.</li>
    <li>Cluster 1 has only one neighborhood that has a bus line and swin school which could suggest many children around who attend K-12 school. </li>
    <li>Cluster 2 also has only one neighborhood that has a French restaurant and a park which could suggest an elderly community.</li>
    <li>Cluster 3 has 31 neighborhoods suggesting almost all of the neighborhoods in Toronto are similar to this group. The top venues suggest that middle aged personal live in this area that like to socialize as evidenced by the prevalance of coffee shops and Cafes</li>
</ol>